In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import time
import os
from tqdm import tqdm

In [2]:
fpath = "../../FinanceData/DB/Chart/Not_Adjusted"
topath = "../../FinanceData/DB/AllFeatures"
temppath = "../../FinanceData/DB/AllFeatures/temp"

In [3]:
ldir = os.listdir(fpath)
files = [fname for fname in ldir if os.path.isfile(os.path.join(fpath, fname))]
data = {fname.removesuffix('.csv'):pd.read_csv('/'.join([fpath, fname]), dtype={'수정주가구분':object, '수정비율':object}) \
    for fname in tqdm(files, desc="Loading Data")}
for k in data: data[k].index = pd.to_datetime(data[k]['일자'])
print("Finished: Loading Data")

Loading Data: 100%|██████████| 2559/2559 [00:24<00:00, 102.77it/s]


Finished: Loading Data


In [4]:
data['000020']

,일자,시가,고가,저가,종가,거래량,거래대금,수정주가구분,수정비율
일자,,,,,,,,,
2024-03-13,2024-03-13,9620,9640,9430,9470,88026,836,NaN,NaN
2024-03-12,2024-03-12,9560,9690,9550,9610,45521,437,NaN,NaN
2024-03-11,2024-03-11,9550,9630,9540,9550,31962,306,NaN,NaN
2024-03-08,2024-03-08,9570,9620,9490,9530,53075,506,NaN,NaN
2024-03-07,2024-03-07,9730,9740,9540,9550,99042,952,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2006-01-09,2006-01-09,20550,20550,19750,19800,33380,666,NaN,NaN
2006-01-06,2006-01-06,19800,20300,19700,20000,60545,1210,NaN,NaN
2006-01-05,2006-01-05,19500,19900,19400,19600,67710,1330,NaN,NaN


In [4]:
def lag(df, col_name, lag_n):
    lags = pd.DataFrame(index=df.index)
    for i in range(1, lag_n+1): lags[f'{col_name}_lag_{i}'] = df[col_name].shift(-i)
    return lags

In [16]:
def feature(df:pd.DataFrame):
    fts = pd.DataFrame(index=df.index)
    fts['rate'] = ((df['종가'] - df['종가'].shift(-1)) / df['종가'].shift(-1)) * 100
    fts = pd.concat([fts, lag(fts, 'rate', 10)], axis=1)
    for n in [5,20,60,120]:
        fts[f'ma{n}'] = df['종가'].iloc[::-1].rolling(window=n).mean().iloc[::-1]
        fts[f'ma{n}disp'] = df['종가'] / fts[f'ma{n}'] * 100
        fts = pd.concat([fts, lag(fts, f'ma{n}disp', 5)], axis=1)
    
    # rounding
    fts = fts.round(4)
    #making y
    fts['y'] = fts['rate'].shift(1)
    return pd.concat([df, fts], axis=1)
        
feature(data['000020'])

,일자,시가,고가,저가,종가,거래량,거래대금,수정주가구분,수정비율,rate,...,ma60disp_lag_4,ma60disp_lag_5,ma120,ma120disp,ma120disp_lag_1,ma120disp_lag_2,ma120disp_lag_3,ma120disp_lag_4,ma120disp_lag_5,y
일자,,,,,,,,,,,,,,,,,,,,,
2024-03-13,2024-03-13,9620,9640,9430,9470,88026,836,NaN,NaN,-1.4568,...,96.0201,97.7212,9824.7500,96.3892,97.7495,97.0792,96.8209,96.9847,98.6794,NaN
2024-03-12,2024-03-12,9560,9690,9550,9610,45521,437,NaN,NaN,0.6283,...,97.7212,97.7490,9831.2500,97.7495,97.0792,96.8209,96.9847,98.6794,98.6568,-1.4568
2024-03-11,2024-03-11,9550,9630,9540,9550,31962,306,NaN,NaN,0.2099,...,97.7490,98.0109,9837.3333,97.0792,96.8209,96.9847,98.6794,98.6568,98.8498,0.6283
2024-03-08,2024-03-08,9570,9620,9490,9530,53075,506,NaN,NaN,-0.2094,...,98.0109,97.8606,9842.9167,96.8209,96.9847,98.6794,98.6568,98.8498,98.6276,0.2099
2024-03-07,2024-03-07,9730,9740,9540,9550,99042,952,NaN,NaN,-1.7490,...,97.8606,98.7008,9846.9167,96.9847,98.6794,98.6568,98.8498,98.6276,99.4167,-0.2094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-01-09,2006-01-09,20550,20550,19750,19800,33380,666,NaN,NaN,-1.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0202
2006-01-06,2006-01-06,19800,20300,19700,20000,60545,1210,NaN,NaN,2.0408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0000
2006-01-05,2006-01-05,19500,19900,19400,19600,67710,1330,NaN,NaN,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0408


In [14]:
def save(temp=False):
    for k in tqdm(data.keys(), desc="Data Saving"):
        feature(data[k]).to_csv(f"{topath if not temp else temppath}/{k}.csv", index=False)

In [17]:
save(temp=False)

Data Saving:   0%|          | 0/2559 [00:00<?, ?it/s]

Data Saving: 100%|██████████| 2559/2559 [05:20<00:00,  7.99it/s]
